# Purpose:
- To query lims data.
- Until CO docDB search and metadata are well established (e.g., which sessions are drifting gratings?)
# Pre-requisites:
- AllenSDK and visual behavior
# Misc:
- Copied from jinho_data_analysis/pilots/231106_GCaMP8s_vs_8m_from_lims.ipynb

In [3]:
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
from brain_observatory_qc.data_access import from_lims

from visual_behavior import database as db
from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache as bpc


In [2]:
# helper functions

def get_lims_results(osid):
    query = '''
    SELECT 
    os.id AS osid, 
    oe.id AS oeid,
    st.acronym AS region,
    oe.calculated_depth, imd.depth,
    os.stimulus_name AS session_type,
    os.date_of_acquisition,
    equipment.name AS equipment_name

    FROM ophys_sessions os
    
    JOIN ophys_experiments oe
    ON oe.ophys_session_id = os.id
    JOIN structures st ON st.id = oe.targeted_structure_id
    JOIN imaging_depths imd ON imd.id = oe.imaging_depth_id
    JOIN equipment ON equipment.id = os.equipment_id
    WHERE os.id = {}
    '''.format(osid)
    lims_results = db.lims_query(query)
    return lims_results

In [4]:
cache = bpc.from_lims()
table = cache.get_ophys_experiment_table(passed_only=False)

# Manual log of mouse IDs

In [24]:
table.session_type.unique()

array(['OPHYS_2_images_A_passive', 'OPHYS_3_images_A',
       'OPHYS_7_receptive_field_mapping', 'OPHYS_4_images_B',
       'OPHYS_5_images_B_passive', 'OPHYS_6_images_B', 'OPHYS_1_images_A',
       'STAGE_1', 'TRAINING_0_gratings_autorewards_15min', 'STAGE_0',
       'PASSIVE_OPHYS_PROD_EXP_DAY13_15',
       'PASSIVE_OPHYS_PROD_UNEXP_DAY16_18', '5_images_a_ophys',
       'OPHYS_1_images_B', 'OPHYS_5_images_A_passive', 'OPHYS_3_images_B',
       'OPHYS_6_images_A', 'OPHYS_4_images_A', 'OPHYS_2_images_B_passive',
       'SNR_2', 'SNR_3', 'SNR_4', 'SNR_6', 'SNR_5', 'SNR_7', 'SNR_8',
       'SNR_0', 'SNR_1', 'SNR_9', 'TRAINING_1_gratings',
       'TRAINING_2_gratings_flashed', 'TRAINING_3_images_A_10uL_reward',
       'TRAINING_4_images_A_training', 'TRAINING_5_images_A_epilogue',
       'TRAINING_5_images_A_handoff_ready', 'OPHYS_6_images_H',
       'OPHYS_2_images_G_passive', 'OPHYS_3_images_G', 'OPHYS_4_images_H',
       'OPHYS_1_images_G', 'OPHYS_5_images_H_passive', 'square_stim',
  

In [25]:
table.query('session_type == "STAGE_1"').full_genotype.unique()

array(['Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt',
       'Cux2-CreERT2/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/wt',
       'Rbp4-Cre_KL100/wt;Camk2a-tTA/wt;Ai93(TITL-GCaMP6f)/wt',
       'Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt',
       'Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai195(TIT2L-GC7s-ICF-IRES-tTA2)-hyg/wt',
       'Slc32a1-IRES-Cre/wt;Oi4(TIT2L-jGCaMP8s-RiboL1-WPRE-ICL-IRES-tTA2-WPRE)/wt',
       'Vipr2-IRES2-Cre-neo/wt',
       'Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Ai210(TITL-GC7f-ICF-IRES-tTA2)-hyg/wt',
       'Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt',
       'Gad2-IRES-Cre/wt;Slc32a1-T2A-FlpO/wt;Oi3(TIT2L-jGCaMP8m-WPRE-ICL-IRES-tTA2-WPRE)/wt',
       'Snap25-IRES2-Cre/wt;Oi4(TIT2L-jGCaMP8s-RiboL1-WPRE-ICL-IRES-tTA2-WPRE)/wt',
       'Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt',
       'Pvalb-IRES-Cre/wt;Ai162(TIT2L-GC6s-ICL-tTA2)/wt', 'wt/wt'],
      dtype=object)

In [27]:
table.query('session_type == "STAGE_1" and full_genotype=="Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt"').mouse_id.unique()

array([687000, 722885, 726465, 693996, 692478, 721292, 717824, 687001])

In [39]:
osid_counts = table.query('session_type == "STAGE_1" and full_genotype=="Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt"').groupby(['ophys_session_id']).equipment_name.count()
# choose session ids with 2 equipment names
osids = osid_counts[osid_counts==2].index
osids


Int64Index([1299462513, 1299688535, 1299958728, 1300188114, 1300974860,
            1301716302, 1304601605, 1304806360, 1305037814, 1305055399,
            1305548254, 1305964574, 1306155299, 1307507669],
           dtype='int64', name='ophys_session_id')

In [41]:
table.query('session_type == "STAGE_1" and full_genotype=="Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2)/wt"').groupby(['ophys_session_id']).equipment_name.count()

ophys_session_id
1299462513    2
1299688535    2
1299958728    2
1300188114    2
1300974860    2
1301716302    2
1303235340    3
1304601605    2
1304806360    2
1305037814    2
1305055399    2
1305548254    2
1305964574    2
1306155299    2
1307507669    2
1363637922    8
1363894265    6
1364262545    8
1364404385    8
1364421327    8
1364445033    5
1364674737    8
1364964612    4
1365199511    8
1369937460    6
1370479387    6
1370949488    4
Name: equipment_name, dtype: int64

In [40]:
table.query('ophys_session_id in @osids').mouse_id.unique()

array([687000, 693996, 692478, 687001])

In [42]:
table.query('ophys_session_id == 1303235340')

,equipment_name,donor_id,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,foraging_id,cre_line,...,session_name,isi_experiment_id,imaging_depth,targeted_structure,published_at,date_of_acquisition,session_type,experience_level,passive,image_set
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
1303663406,MESO.2,1283840034,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,687000,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,134.0,ad69df47-6f6e-4220-8fc9-3f237e5b05d0,Slc32a1-IRES-Cre,...,20231012_spark_687000_stage1,1286961061,75,VISp,NaT,2023-10-12 19:46:20.029,STAGE_1,None,False,N/A
1303663410,MESO.2,1283840034,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,687000,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,134.0,ad69df47-6f6e-4220-8fc9-3f237e5b05d0,Slc32a1-IRES-Cre,...,20231012_spark_687000_stage1,1286961061,200,VISp,NaT,2023-10-12 19:46:20.029,STAGE_1,None,False,N/A
1303663412,MESO.2,1283840034,Slc32a1-IRES-Cre/wt;Oi1(TIT2L-jGCaMP8s-WPRE-IC...,687000,Oi1(TIT2L-jGCaMP8s-WPRE-ICL-IRES-tTA2),[Slc32a1-IRES-Cre],M,134.0,ad69df47-6f6e-4220-8fc9-3f237e5b05d0,Slc32a1-IRES-Cre,...,20231012_spark_687000_stage1,1286961061,352,VISp,NaT,2023-10-12 19:46:20.029,STAGE_1,None,False,N/A


In [45]:
mids_ribo_aav_local = [719364, 726087, 719363]
mids_ribo_aav_ro = [730929, 730932, 730933]
mids_ribo_aav_icv = []
mids_snap25_oi4_dox = [726433]
mids_slc32a1_oi4 = [724567, 729088]
mids_slc17a7_oi4 = []
mids_slc32a1_oi1 = [687000, 693996, 692478, 687001] # need to exclude osid 1303235340 
mids_slc17a7_oi1_dox = [733794]
mids_cux2_oi1 = []

In [9]:
table.mouse_id = table.mouse_id.astype(int)

In [11]:
table.query('mouse_id in @mids_ribo_aav_local').session_type.unique()

array(['OPHYS_2_images_A_passive', 'STAGE_1'], dtype=object)

In [13]:
table.keys()

Index(['equipment_name', 'donor_id', 'full_genotype', 'mouse_id',
       'reporter_line', 'driver_line', 'sex', 'age_in_days', 'foraging_id',
       'cre_line', 'indicator', 'session_number',
       'prior_exposures_to_session_type', 'prior_exposures_to_image_set',
       'prior_exposures_to_omissions', 'ophys_session_id',
       'behavior_session_id', 'ophys_container_id', 'project_code',
       'container_workflow_state', 'experiment_workflow_state', 'session_name',
       'isi_experiment_id', 'imaging_depth', 'targeted_structure',
       'published_at', 'date_of_acquisition', 'session_type',
       'experience_level', 'passive', 'image_set'],
      dtype='object')

In [51]:
lims_path_info = pd.DataFrame()

temp = table.query('mouse_id in @mids_ribo_aav_local and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'ribo_aav_local'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_ribo_aav_ro and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'ribo_aav_ro'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_snap25_oi4_dox and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'snap25_oi4_dox'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_slc32a1_oi4 and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'slc32a1_oi4'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_slc17a7_oi1_dox and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'slc17a7_oi1_dox'

lims_path_info = pd.concat([lims_path_info, temp])

temp = table.query('mouse_id in @mids_slc32a1_oi1 and session_type=="STAGE_1"').set_index('ophys_session_id')[['mouse_id', 'date_of_acquisition']].drop_duplicates()
temp['lims_path'] = temp.index.map(lambda x: from_lims.get_session_h5_filepath(x).parent)
temp['gcamp'] = 'slc32a1_oi1'
temp = temp.drop(1303235340)

lims_path_info = pd.concat([lims_path_info, temp])

In [52]:
lims_path_info

,mouse_id,date_of_acquisition,lims_path,gcamp
ophys_session_id,,,,
1363436007,726087,2024-05-01 16:00:28.374,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1363660075,726087,2024-05-02 17:40:50.123,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1373890769,726087,2024-06-17 16:01:47.183,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1374103167,726087,2024-06-18 17:34:38.386,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1347962182,719363,2024-04-26 16:40:02.748,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1347758257,719363,2024-04-25 16:17:26.610,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1369711200,719363,2024-05-30 15:57:27.365,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1370438358,719363,2024-06-03 15:24:46.802,\\allen\programs\mindscope\production\learning...,ribo_aav_local
1344380721,719364,2024-04-11 18:36:21.565,\\allen\programs\mindscope\production\learning...,ribo_aav_local


In [59]:
lims_path_info.date_of_acquisition.values[0]
# change datetime64 to string with format 'YYYY-MM-DD_HH-MM-SS'
lims_path_info.date_of_acquisition = lims_path_info.date_of_acquisition.dt.strftime('%Y-%m-%d_%H-%M-%S')

In [60]:
save_dir = Path(r'\\allen\programs\mindscope\workgroups\learning\pilots\GCaMP8')
save_fn = save_dir / 'stage1_data_240703.csv'
lims_path_info.to_csv(save_fn)